In [197]:
import sys
import os
import glob
import csv
sys.path.append('/usr/local/lib/python2.7/dist-packages')
import cv2

from scipy import ndimage as ndi

from skimage import color, io, exposure, feature, filters, measure, data
from skimage.filters import threshold_otsu, gaussian_filter, gabor_kernel
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float, img_as_ubyte

import numpy as np
from numpy import cos, sin, conjugate, sqrt

from math import atan2

from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['font.size'] = 9

import ttk
from Tkinter import *
from PIL import Image, ImageTk

In [198]:
# Returns the filepaths and filenames of all those files in 'directory' ending in '.fmt'
# Currently hard-coded
def get_path(directory, fmt):
    a = glob.glob(directory + '/*.' + fmt)
    b = [x.split('/')[-1].split('.')[0] for x in a]
    return zip(a,b)

In [199]:
def get_init_image(fname, all_paths):
    if fname == '':
        ind=0
    else:
        ind = all_paths.index(fname)+1
    img = io.imread(all_paths[ind][0])
    current = all_paths[ind][0]
    gray = color.rgb2gray(img)
    
    # The compactness value maybe reduced in order to improve the segmentation
    segments_slic = slic(img, n_segments=200, compactness=5, sigma=1, convert2lab = True)
    segments_slic = segments_slic + 1  # So that no labelled region is 0 and ignored by regionprops
    regions = measure.regionprops(segments_slic)
    return img, gray, current, regions, segments_slic, mark_boundaries(img, segments_slic,[0,0,0])


In [200]:
def getImageTk(img):
    img = img_as_ubyte(img)
    i = Image.fromarray(img)
    return ImageTk.PhotoImage(i)

In [201]:
# Main begins

print "Welcome"

# Get paths to all images
paths = get_path(os.getcwd(),'tiff')

index = 0 # index denotes the row number in the csv file which corresponds to the start of the first SP region
filepath = paths[0][0] # filepath is the path to the corresponding image whose first SP starts from 'index' row'

if not os.path.isfile("labeled_data.csv"):
    print "Data not found. Create a new CSV file named labeled_data.csv and index_filename.csv."

Welcome


In [202]:
last_index = 0            # The last index indicates start point available. 0 if no previous data has been written
last_fname = '' 

# Read important stuff and close
if os.path.getsize('labeled_data.csv') > 0: # If it is not empty
    with open("labeled_data.csv",'rb') as labeled_data, open("index_filename.csv",'rb') as index_fname:
        ifreader = csv.reader(index_fname, delimiter = ",")
        for row in ifreader:
            print ', '.join(row)
            old = row
        last_index = old[0]
        last_fname = old[1]
        print last_index, last_fname

In [203]:
current_img, current_gray, current_img_path, regions, segments, boundary_img = get_init_image(last_fname,paths)
marked_img = boundary_img.copy() # Shown in window 2 of GUI

In [204]:
current_highlight = 1
# This function highlights a particular Superpxel region of the image
def color_region(region_image, segments, highlighted_region_num):
    if highlighted_region_num < segments.min() or highlighted_region_num > segments.max():
        print "No region corresponding to this region number exists"
        return region_image
    new_img = boundary_img.copy()
    # Highlight the region with a color
    flags = segments==highlighted_region_num
    new_img[flags] = (boundary_img[flags] + [1.0,1.0,0.0])/2.0
    return new_img

# This function marks(labels) a particular Superpxel region of the image
def mark_region(region_img, segments, highlighted_region_num, positive):
    if highlighted_region_num < segments.min() or highlighted_region_num > segments.max():
        print "No region corresponding to this region number exists"
        return region_image
    global marked_img
    # Mark the region with white if positive, black if not positive
    flags = segments==highlighted_region_num
    if positive:
        marked_img[flags] = [1,1,1]
    else:
        marked_img[flags] = [0,0,0]
    return marked_img

In [205]:
# Callbacks defined 
# ------------ Scrolling Callbacks ----------
def callback_Next_SP():
    global current_highlight
    if current_highlight < segments.max():
        current_highlight += 1
        next_img = getImageTk(color_region(boundary_img,segments,current_highlight))
        label_o.configure(image = next_img)
        label_o.image = next_img

def callback_Previous_SP():
    global current_highlight
    if current_highlight > segments.min():
        current_highlight -= 1
        next_img = getImageTk(color_region(boundary_img,segments,current_highlight))
        label_o.configure(image = next_img)
        label_o.image = next_img
def callback_Prev_Img():
    if current_img_path == paths[0][0]:
        button_prev_img.configure()
def callback_Next_Img():
    button_prev_img.configure()

# ------------ Labelling Callbacks ----------
def callback_Positive():
    # Some Magic
    next_img = getImageTk(mark_region(marked_img,segments,current_highlight,True))
    label_g.configure(image = next_img)
    label_g.image = next_img
    callback_Next_SP()
def callback_Negative():
    # Some Magic
    next_img = getImageTk(mark_region(marked_img,segments,current_highlight,False))
    label_g.configure(image = next_img)
    label_g.image = next_img
    callback_Next_SP()
def callback_Pos_Border():
    # Some Magic
    next_img = getImageTk(mark_region(marked_img,segments,current_highlight,True))
    label_g.configure(image = next_img)
    label_g.image = next_img
    callback_Next_SP()
def callback_Neg_Border():
    # Some Magic
    next_img = getImageTk(mark_region(marked_img,segments,current_highlight,False))
    label_g.configure(image = next_img)
    label_g.image = next_img
    callback_Next_SP()

In [206]:
# Write data onto csv files

with open("labeled_data.csv",'a') as labeled_data, open("index_filename.csv",'a') as index_fname:
    ldwriter = csv.writer(labeled_data, delimiter = ",")
    ifwriter = csv.writer(index_fname, delimiter = ",")
    
    if not os.path.getsize('index_filename.csv') > 0: # For first time creation
        ifwriter.writerow([0,''])
             
        
    root = Toplevel() # Change this to Tk() instead of Toplevel() to prevent 2 windows from opening
    root.title("Super-Pixel Labelling")

    # frame1 is for displaying the 2 images
    frame1 = ttk.Frame(root)
    frame1.grid(column=1, row=1)

    # Displaying the original picture with the regions marked
    ttk.Label(frame1, text="Original").grid(column=1, row=1)
    gui_orig_photo = getImageTk(color_region(boundary_img,segments,1))
    label_o = ttk.Label(frame1, image=gui_orig_photo)
    label_o.image = gui_orig_photo
    label_o.grid(column = 1, row = 2)

    # Shows the final bitmap with Sclera region marked in white and rest in black
    ttk.Label(frame1, text="Region").grid(column=2, row=1)
    regions_marked_img = getImageTk(marked_img)
    label_g = ttk.Label(frame1, image=regions_marked_img)
    label_g.image = regions_marked_img
    label_g.grid(column = 2, row = 2)

    # frame2 is for the labelling buttons and for scrolling between images
    frame2 = ttk.Frame(root)
    frame2.grid(column=1, row=2)

    ttk.Button(frame2, text="Positive", command = callback_Positive).grid(column=1, row=1)
    ttk.Button(frame2, text="Pos. Border", command = callback_Pos_Border).grid(column=3, row=1)
    ttk.Button(frame2, text="Prev SuperPixel", command = callback_Previous_SP).grid(column=2, row=2)
    ttk.Button(frame2, text="Save Image Data", command = callback_Positive).grid(column=4, row=2)

    ttk.Button(frame2, text="Negative", command = callback_Negative).grid(column=5, row=1)
    ttk.Button(frame2, text="Neg. Border", command = callback_Neg_Border).grid(column=7, row=1)
    ttk.Button(frame2, text="Next SuperPixel", command = callback_Next_SP).grid(column=6, row=2)


    button_prev_img = ttk.Button(frame2, text="Prev Img", command = callback_Prev_Img)
    button_prev_img.grid(column=1, row=3)
    ttk.Button(frame2, text="Done", command = callback_Positive).grid(column=4, row=3)
    ttk.Button(frame2, text="Next Img", command = callback_Next_Img).grid(column=7, row=3)


    for child in frame1.winfo_children(): child.grid_configure(padx=5, pady=5)

    root.mainloop()

In [13]:
a = get_path(os.getcwd(),'tiff')
a

[('/home/ankush/Sclera/src_py/1.tiff', '1'),
 ('/home/ankush/Sclera/src_py/3.tiff', '3'),
 ('/home/ankush/Sclera/src_py/2.tiff', '2'),
 ('/home/ankush/Sclera/src_py/4.tiff', '4'),
 ('/home/ankush/Sclera/src_py/5.tiff', '5')]

In [14]:
paths

[('/home/ankush/Sclera/src_py/1.tiff', '1'),
 ('/home/ankush/Sclera/src_py/3.tiff', '3'),
 ('/home/ankush/Sclera/src_py/2.tiff', '2'),
 ('/home/ankush/Sclera/src_py/4.tiff', '4'),
 ('/home/ankush/Sclera/src_py/5.tiff', '5')]

In [15]:
os.path.isfile("6.tiff")

False

In [16]:
with open('eggs.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])
    spamwriter.writerow(['Spam', 'ok Spam', 'ugly Spam'])

In [22]:
os.path.getsize('index_filename.csv')

4

In [18]:
with open('eggs.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print ', '.join(row)
        old = row

Spam,Spam,Spam,Spam,Spam,Baked, Beans
Spam,Lovely, Spam,Wonderful, Spam
Spam,ok, Spam,ugly, Spam
